In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import dateutil.relativedelta
import math
import pygsheets
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from adaptation import ExcelReview

In [2]:
token_new_bi = 'HtdQ1pqETq'
token_cosm_bi = 'lpLLZsHACx'

current_date = datetime.today()
c_day = current_date.day
c_month = current_date.month
c_year = current_date.year

yesterday_date = current_date - dateutil.relativedelta.relativedelta(days=1)
y_day = yesterday_date.day
y_month = yesterday_date.month
y_year = yesterday_date.year


seven_month_date = current_date - dateutil.relativedelta.relativedelta(month=7)
s_day = seven_month_date.day
s_month = seven_month_date.month
s_year = seven_month_date.year


def first_month_day(table, sheet, merge_col, other_cols):
    if y_day==1:
        table.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', sheet, 'a1')
    else:
        table.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', sheet, merge_col, other_cols)

In [ ]:
cat_str = "Баннер, АВИТО (Ручная подача), Авито (Покупка услуг), Авито (руч. подача), Авито - поднятие объявлений, Авито реклама, Баннер, Баннеры, Баннеры на колышках, Баннеры-г.Челябинск, Баннеры-ручное списание, Брендовая расклейка, Визитки, Визитки г. Пыть-Ях, Визитки-наклейки, Выгрузка за объявления на МЛСН, Выгрузка на сайты, Выгрузка объявлений на сайты (автовыгрузка), Дополнительный статус на Авито, За Баннеры, За баннеры, За визитки, За поднятие объявления на сайте Авито, За рекламу, За рекламу на АВИТО, За услуги расклейщика, Заказ баннера, Заказ баннеров, Заказ баннеров/фолий, Заказ визиток, Изготовление Баннеров-г.Когалым, Изготовление баннеров, Индивидуальная реклама, Использование баннера, Клей для расклейки, Листовки, Листовки , Маркетинг, Монтаж баннера, Наклейка на окно , Наклейка-визитка, Наклейки, Наклейки на авто, Наклейки на автомобиль, Наклейки на стекла, Объявление Авито, Объявление в газеты, Объявление на сайт N1, Объявление на сайт Авито , Объявление на сайты, Объявление на сайты , Объявление на сайты АвитоPro, Объявления в газеты, Объявления на АВИТО, Объявления на сайт 63.ru, Объявления на сайт АВИТО, Объявления на сайт Авито, Объявления на сайт Авито (ручная выгрузка), Объявления на сайт Из рук в руки, Объявления на сайт Сибдом, Объявления на сайт Циан (ручная выгрузка), Объявления на сайт ЯКТ, Объявления на сайте Avito, Объявления на сайты, Оплата размещения на авито, Оплата расклейщикам, Печать расклейки, Платная выгрузка, Применение платного статуса на Авито, Размещения объявлений на сайт авито, Расклейка, Расклейка объявлений , Расклейщик, Реклама, Реклама на АВИТО, Реклама на Авито, Рекламные листовки, Рекламные расходы, Служба сервиса: расклейка, Служба сервиса: фотографирование, Создание баннеров, Создание визиток, Создание наклеек на авто, Списание за баннер, Списание за визитки, Списание за объявления на сайт ЕМЛС, Списание за расклейку, Списание за рекламу на Авито, Списание на рекламу, Списание с бонусного счета (Авито), Строчная реклама, Услуги расклейщика, ЦИАН (ручная подача), Циан (ручная подача), баннеры, визитка в авто, визитки, доп.статусы на Авито, заказ баннера, заказ баннеров, заказ визиток, заказ наклеек, монтаж баннера, наклейка на автомобиль, наклейка на стекло, оплата баннеров, печать визиток, печать расклейки, платный статус, размещение объявлений в газету, расклейка"
cat_list = cat_str.split(', ')

req_zatrati_rekl = {
    "token": token_new_bi,
    "report_id": 393,
    "fields": ["city_name", "fio", "summ", "payment_date"],
    "filter_query":[
        {
            "field": "payment_date",
            "values": [f"{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        },
        {
            "field": "cat_name",
            "values": cat_list
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_req_zatrati_rekl(table):
    table['payment_date'] = pd.to_datetime(table['payment_date'])
    table['date'] = table['payment_date'].apply(lambda x: x.strftime('%Y-%m-%d'))

    table['summ'] = table['summ'].astype(float)
    end_table = table.pivot_table('summ', ['city_name', 'fio'], 'date', aggfunc="sum")
    end_table = end_table.fillna(0)
    #end_table['summing'] = end_table.sum(axis=1)
    end_table = end_table.reset_index()
    
    date_column = f"{y_year}-{y_month}-{y_day}"
    end_table[date_column] = end_table[date_column].astype(str)
    end_table[date_column] = end_table[date_column].apply(lambda x: x.replace('.', ','))

    return end_table

req_zatrati_rekl = ExcelReview(req_zatrati_rekl)
req_zatrati_rekl.df = make_table_req_zatrati_rekl(req_zatrati_rekl.get_table())

first_month_day(req_zatrati_rekl, 'РекламаДень', 'fio', ['city_name'])
#req_zatrati_rekl.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'РекламаДень', 'a1')
#req_zatrati_rekl.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'РекламаДень', 'fio', ['city_name'])

In [4]:
del req_zatrati_rekl

In [ ]:
zayv_type = 'Дистанционная покупка, Новостройки, Покупка вторичная, Покупка гаражи, Покупка загородной, Покупка загородной элитной недвижимости , Покупка коммерческой, Покупка элитная'
zayv_type = zayv_type.split(', ')

zayv_status = 'deposit, free, open'
zayv_status = zayv_status.split(', ')

zayv_reason = ', , Другое, Клиент отказался от фотографирования, На контроле у риэлтора, Нет ответа, Объект на задатке, Объект продан, Объект удален, Отмена, Продажа объекта приостановлена, Риэлтор отказался от фотографирования, Риэлтор самостоятельно сфотографировал объект, Семейные обстоятельства, выбран неверный город, консультация, купили/продали, не актуально (без причин), не доступен, не отвечает, нереальная потребность, нет подходящих объектов, объект находится далеко, отказались из-за комиссии, отложена, думают, передано другому риэлтору, передумали, по давности, сдали/сняли, сняли с другим АН'
zayv_reason = zayv_reason.split(', ')


zayv_na_pokup = {
    'token': token_new_bi,
    'report_id': 242,
    'fields': ['realtor', 'u_city', 'ticket_id'],
    'filter_query': [
        {
            'field': 'type_name',
            'values': zayv_type
        },
        {
            'field': 'status',
            'values': zayv_status
        }, 
        {
            'field': 'reason',
            'values': zayv_reason
        },
        {
            'field': 'realtor_start_date',
            "values": [f"{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}"],
            "type": "date"
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_zayv_na_pokup(table):
    table['ticket_id'] = table['ticket_id'].astype(str)
    
    table = table.pivot_table('ticket_id', ['realtor', 'u_city'], aggfunc='count')
    table = table.reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'ticket_id':date_column})
    
    return table


zayv_na_pokup = ExcelReview(zayv_na_pokup)
zayv_na_pokup.df = make_table_zayv_na_pokup(zayv_na_pokup.get_table())

first_month_day(zayv_na_pokup, 'ЗаявкиНаПокупкуДень', 'realtor', ['u_city'])
#zayv_na_pokup.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиНаПокупкуДень', 'a1')
#zayv_na_pokup.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиНаПокупкуДень', 'realtor', ['u_city'])

In [9]:
del zayv_na_pokup

In [ ]:
city_filter = '--, Abu Dhabi, Alanya, Antalya, Dubai, Northern Cyprus, Pattaya, Phuket, Plzeň, Praha, Romashka, Sharjah, Абакан, Актау, Актобе, Алапаевск, Алексеевка, Алматы, Алушта, Альметьевск, Анапа, Ангарск, Анжеро-Судженск, Арсеньев, Архангельск, Астана, Астрахань, Атырау, Ачинск, Аша, Байконур, Баку, Балашов, Барнаул, Бахчисарай, Белгород, Белогорск, Амурская область, Бердск, Березовский (Кемеровская область), Бийск, Бирюч, Бишкек, Благовещенск, Боровичи, Брянск, Бургас, Валуйки, Вейделевка пгт., Великий Новгород, Верхняя Салда, Владивосток, Волгоград, Волжск, Волжский, Волоконовка пгт., Волхов, Воронеж, Воскресенск, Воткинск, Гатчина, Горно-Алтайск, Горячий Ключ, Грозный, Губкин, Губкинский, Далматово, Дербент, Дзержинск, Дмитров, Домодедово, Другой город, Душанбе, Евпатория, Ейск, Екатеринбург, Ереван, Железногорск, Зеленодольск, Златоуст, Иваново, Ижевск, Ирбит, Иркутск, Ишим, Йошкар-Ола, Казань, Калининград, Калуга, Камышлов, Каспийск, Кемерово, Кириши, Киров, Клин, Ковров, Когалым, Кокшетау, Коломна, Комсомольск-на-Амуре, Короча, Корсаков, Коряжма, Костанай, Кострома, Котлас, Краснодар, Красноярск, Кумертау, Курган, Кызыл, Кыштым, Лангепас, Ленинск-Кузнецкий, Липецк, Луховицы, Лянтор, Майкоп, Махачкала, Междуреченск, Мелеуз, Миасс, Минск, Мичуринск, Можга, Москва, Муравленко, Муром, Набережные Челны, Надым, Назрань, Нальчик, Наро-Фоминск, Невинномысск, Нерчинск, Нефтекамск, Нефтеюганск, Нижневартовск, Нижний Новгород, Нижний Тагил, Новая Уфа, Новодвинск, Новокузнецк, Новороссийск, Новосибирск, Новоуральск, Новый Волгоград, Новый Оскол, Новый Уренгой, Норильск, Ноябрьск, Нягань, Обнинск, Одесса, Озерск, Октябрьский (Башкортостан), Октябрьский (Пермский край), Омск, Омская область, Опочка, Оренбург, Орехово-Зуево, Орёл, Остров, Павлодар, Пенза, Первоуральск, Пермь, Пермь 2, Петрозаводск, Петропавловск, Петропавловск-Камчатский, Печоры, Подольск, Пойковский пгт., Покров, Прокопьевск, Прохладный, Псков, Пыть-Ях, Пятигорск, Ракитное, Раменское, Московская обл., Ростов-на-Дону, Рудный, Рязань, Саки, Салават, Салехард, Самара, Самарканд, Санкт-Петербург, Саранск, Сарапул, Саратов, Сатка, Свободный, Северодвинск, Семей, Симферополь, Смоленск, Сосновоборск, Сочи, Ставрополь, Старый Оскол, Стерлитамак, Судак, Сургут, Сыктывкар, Таганрог, Талица, Тамбов, Тарко-Сале, Ташкент, Тверь, Темрюк, Тобольск, Тольятти, Томск, Троицк, Туапсе, Туймазы, Тула, Туринск, Тюмень, Улан-Удэ, Ульяновск, Уральск, Усинск, Усть-Каменогорск, Уфа, Ухта, Феодосия, Хабаровск, Ханты-Мансийск, Чебаркуль, Челябинск, Череповец, Черкесск, Черногорск, Чита, Шадринск, Шексна, Шымкент, Электросталь, Южно-Сахалинск, Южноуральск, Юрга, Якутск, Ялта, Ялуторовск, Ярославль, Ярцево'
city_filter = city_filter.split(', ')


objects_in_base_active = {
    'token': token_new_bi, 
    'report_id': 347,
    'fields': ['city', 'fio', 'count_object'],
    'filter_query': [
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


objects_in_base_contract = {
    'token': token_new_bi, 
    'report_id': 347,
    'fields': ['city', 'fio', 'count_object_contract'],
    'filter_query': [
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


objects_in_base_flats = {
    'token': token_new_bi, 
    'report_id': 347,
    'fields': ['city', 'fio', 'count_flats'],
    'filter_query': [
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


objects_in_base_cottages = {
    'token': token_new_bi, 
    'report_id': 347,
    'fields': ['city', 'fio', 'count_cottages'],
    'filter_query': [
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_objects_in_base(table, obj_type):
    table = table.fillna(0)

    table[obj_type] = table[obj_type].astype(str)
    table[obj_type] = table[obj_type].apply(lambda x: x.replace('.', ','))

    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={obj_type:date_column})

    return table


objects_in_base_active = ExcelReview(objects_in_base_active)
objects_in_base_active.df = make_table_objects_in_base(objects_in_base_active.get_table(), 'count_object')

first_month_day(objects_in_base_active, 'объекты_в_базе_активный', 'fio', ['city'])
#objects_in_base_active.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_активный', 'a1')
#objects_in_base_active.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_активный', 'fio', ['city'])

objects_in_base_contract = ExcelReview(objects_in_base_contract)
objects_in_base_contract.df = make_table_objects_in_base(objects_in_base_contract.get_table(), 'count_object_contract')

first_month_day(objects_in_base_contract, 'объекты_в_базе_договоры', 'fio', ['city'])
#objects_in_base_contract.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_договоры', 'a1')
#objects_in_base_contract.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_договоры', 'fio', ['city'])

objects_in_base_flats = ExcelReview(objects_in_base_flats)
objects_in_base_flats.df = make_table_objects_in_base(objects_in_base_flats.get_table(), 'count_flats')

first_month_day(objects_in_base_flats, 'объекты_в_базе_вторичный', 'fio', ['city'])
#objects_in_base_flats.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_вторичный', 'a1')
#objects_in_base_flats.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_вторичный', 'fio', ['city'])

objects_in_base_cottages = ExcelReview(objects_in_base_cottages)
objects_in_base_cottages.df = make_table_objects_in_base(objects_in_base_cottages.get_table(), 'count_cottages')

first_month_day(objects_in_base_cottages, 'объекты_в_базе_загородный', 'fio', ['city'])
#objects_in_base_cottages.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_загородный', 'a1')
#objects_in_base_cottages.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'объекты_в_базе_загородный', 'fio', ['city'])


In [11]:
del objects_in_base_active
del objects_in_base_contract
del objects_in_base_flats
del objects_in_base_cottages

In [ ]:
podborka_filter = 'Да(на основе задатка), Да(на основе сделки), Да(настоящая)'
podborka_filter = podborka_filter.split(', ')
podborka_z_type  = 'Новостройки, Покупка вторичной, Покупка загородной, Покупка коммерческой'
podborka_z_type = podborka_z_type.split(', ')

podborki = {
    'token': token_new_bi, 
    'report_id': 513,
    'fields': ['rieltor_city', 'u_fio', 'date_podbor'],
    'filter_query': [
        {
            'field': 'podbor',
            'values': podborka_filter
        },
        {
            'field': 'type',
            'values': podborka_z_type
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_podbor',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_podborki(table):
    table = table.fillna(0)

    table =  table.groupby(['rieltor_city', 'u_fio']).nunique().reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'date_podbor': date_column})
    
    return table


podborki = ExcelReview(podborki)
podborki.df = make_table_podborki(podborki.get_table())

first_month_day(podborki, 'ПодборкиДень', 'u_fio', ['rieltor_city'])
#podborki.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ПодборкиДень', 'a1')
#podborki.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ПодборкиДень', 'u_fio', ['rieltor_city'])

In [17]:
del podborki

In [ ]:
filter_pokaz = 'Не выставлена, Объект не понравился, Объект понравился, Показ состоялся, но отметка не проставлена'
filter_pokaz = filter_pokaz.split(', ')

pokazi_pokupatel = {
    'token': token_new_bi, 
    'report_id': 444,
    'fields': ['city', 'rieltor', 'pokaz_date'],
    'filter_query': [
        {
            'field': 'otmentka',
            'values': filter_pokaz
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'pokaz_date',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_pokazi_pokupatel(table):
    table = table.fillna(0)

    table = table.groupby(['city', 'rieltor']).nunique().reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'pokaz_date': date_column})

    return table


pokazi_pokupatel = ExcelReview(pokazi_pokupatel)
pokazi_pokupatel.df = make_table_pokazi_pokupatel(pokazi_pokupatel.get_table())

first_month_day(pokazi_pokupatel, 'ПоказыДень',  'rieltor', ['city'])
#pokazi_pokupatel.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ПоказыДень', 'a1')
#pokazi_pokupatel.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ПоказыДень', 'rieltor', ['city'])

In [19]:
del pokazi_pokupatel

In [ ]:
etap_filter = 'Звонок ГЛИ: Была консультация с брокером, Звонок ГЛИ: Консультация, Консультация: Дистанционная работа, Консультация: Личная, Консультация: видеоконсультация, Онлайн клиент: консультация, Телефонная консультация'
etap_filter = etap_filter.split(', ')
status_filter = ', active'
status_filter = status_filter.split(', ')
client_filter = ['настоящий клиент']


ipotech_consult = {
    'token': token_new_bi, 
    'report_id': 285,
    'fields': ['r_city_name', 'realtor', 'started'],
    'filter_query': [
        {
            'field': 'action_name',
            'values': etap_filter
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'test_realy_client',
            'values': client_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'started',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_ipotech_consult_table(table):
    table.fillna(0)

    table = table.groupby(['r_city_name', 'realtor']).nunique().reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'started': date_column})

    return table


ipotech_consult = ExcelReview(ipotech_consult)
ipotech_consult.df = make_ipotech_consult_table(ipotech_consult.get_table())

first_month_day(ipotech_consult, 'ИКДень', 'realtor', ['r_city_name'])
#ipotech_consult.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ИКДень', 'a1')
#ipotech_consult.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ИКДень', 'realtor', ['r_city_name'])

In [ ]:
del ipotech_consult

In [ ]:
etap_filter = 'Заявка(и) в банк(и)'
etap_filter = etap_filter.split(', ')
status_filter = ', active'
status_filter = status_filter.split(', ')
client_filter = 'настоящий клиент'
client_filter = client_filter.split(', ')


zayv_ipotek_bank = {
    'token': token_new_bi, 
    'report_id': 285,
    'fields': ['r_city_name', 'realtor', 'start_date', 'started'],
    'filter_query': [
        {
            'field': 'action_name',
            'values': etap_filter
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'test_realy_client',
            'values': client_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'started',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_zayv_ipotek_bank(table):
    table = table.fillna(0)

    table = table.groupby(['realtor', 'r_city_name', 'start_date']).count().reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'started': date_column})
    
    return table


zayv_ipotek_bank = ExcelReview(zayv_ipotek_bank)
zayv_ipotek_bank.df = make_table_zayv_ipotek_bank(zayv_ipotek_bank.get_table())

first_month_day(zayv_ipotek_bank, 'ЗаявкиИпотекиДень', 'realtor', ['r_city_name', 'start_date'])
#zayv_ipotek_bank.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗаявкиИпотекиДень', 'a1')
#zayv_ipotek_bank.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ИКДень', 'realtor', ['r_city_name', 'start_date'])

In [ ]:
del zayv_ipotek_bank

In [ ]:
zadatok_status = 'В работе, Сделка'
zadatok_status = zadatok_status.split(', ')


zadatki_table = {
    'token': token_new_bi, 
    'report_id': 257,
    'fields': ['realtor_city', 'u_fio', 'u_start_date', 'date_deposit'],
    'filter_query': [
        {
            'field': 'status_deposit',
            'values': zadatok_status
        },
        {
            'field': 'u_start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_deposit',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


def make_table_zadatki_table(table):
    table = table.fillna(0)

    table = table.groupby(['realtor_city', 'u_fio', 'u_start_date']).count().reset_index()
    date_column = f"{y_year}-{y_month}-{y_day}"
    table = table.rename(columns={'date_deposit': date_column})

    return table


zadatki_table = ExcelReview(zadatki_table)
zadatki_table.df = make_table_zadatki_table(zadatki_table.get_table())

first_month_day(zadatki_table, 'ЗадаткиДень', 'u_fio', ['realtor_city', 'u_start_date'])
#zadatki_table.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗадаткиДень', 'a1')
#zadatki_table.load_by_day('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'ЗадаткиДень', 'u_fio', ['realtor_city', 'u_start_date'])

In [ ]:
del zadatki_table

In [22]:
zadatok_status = 'В работе, Сделка'
zadatok_status = zadatok_status.split(', ')


buy_sell = {
    'token': token_cosm_bi, 
    'report_id': 257,
    'fields': ['realtor_city', 'u_fio', 'operation', 'date_deposit'],
    'filter_query': [
        {
            'field': 'status_deposit',
            'values': zadatok_status
        },
        {
            'field': 'u_start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_deposit',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}


In [ ]:
buy_sell = ExcelReview(buy_sell)
buy_sell.get_table()

In [17]:
city_filter = 'Все кроме пустых'
city_filter = city_filter.split(', ')
zayv_filter = 'Открыта'
zayv_filter = zayv_filter.split(', ')
life_status = 'Загружен, Зарегистрирован, Обновлен'
life_status = life_status.split(', ')


bron_v_rabote ={
    'token': token_cosm_bi, 
    'report_id': 741,
    'fields': ['city', 'agent_name'],
    'filter_query': [
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'ticket_status',
            'values': zayv_filter
        },
        {
            'field': 'doc_state_name',
            'values': life_status
        }
    ],
    "rows": 10000,
    "start": 0
}

In [14]:
city_filter = 'Тюмень'
city_filter = city_filter.split(', ')
sotr_type = 'agent'
sotr_type = sotr_type.split(', ')
sotr_status = 'active, delete'
sotr_status = sotr_status.split(', ')
vhod_status = 'Успешно'
vhod_status = vhod_status.split(', ')


zahodi_v_ries_tymen = {
    'token': token_cosm_bi, 
    'report_id': 302,
    'fields': ['city_name', 'fio', 'login_date'],
    'filter_query': [
        {
            'field': 'city_name',
            'values': city_filter
        },
        {
            'field': 'u_type',
            'values': sotr_type
        },
        {
            'field': 'user_status',
            'values': sotr_status
        },
        {
            'field': 'success',
            'values': vhod_status
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'login_date',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Челябинск'
city_filter = city_filter.split(', ')
sotr_type = 'agent'
sotr_type = sotr_type.split(', ')
sotr_status = 'active, delete'
sotr_status = sotr_status.split(', ')
vhod_status = 'Успешно'
vhod_status = vhod_status.split(', ')


zahodi_v_ries_chelyab = {
    'token': token_cosm_bi, 
    'report_id': 302,
    'fields': ['city_name', 'fio', 'login_date'],
    'filter_query': [
        {
            'field': 'city_name',
            'values': city_filter
        },
        {
            'field': 'u_type',
            'values': sotr_type
        },
        {
            'field': 'user_status',
            'values': sotr_status
        },
        {
            'field': 'success',
            'values': vhod_status
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'login_date',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')


reklama_cosm = {
    'token': token_cosm_bi, 
    'report_id': 18,
    'fields': ['city_rielt', 'rieltor_fio', 'max_start_date'],
    'filter_query': [
        {
            'field': 'city_rielt',
            'values': city_filter
        },
        {
            'field': 'started_date_rielt',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'max_start_date',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
zayv_type = 'Покупка'
zayv_type = zayv_type.split(', ')
zayv_status = 'В работе, На задатке'
zayv_status = zayv_status.split(', ')


zayv_na_pokup_cosm = {
    'token': token_cosm_bi, 
    'report_id': 5,
    'fields': ['rieltor_city', 'rieltor_fio'],
    'filter_query': [
        {
            'field': 'rieltor_city',
            'values': city_filter
        },
        {
            'field': 'ticket_type',
            'values': zayv_type
        },
        {
            'field': 'ticket_status',
            'values': zayv_status
        },
        {
            'field': 'success',
            'values': vhod_status
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'ticket_created',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
rieltor_status = 'Активен'
rieltor_status = rieltor_status.split(', ')
zayv_status = 'В работе, На задатке'
zayv_status = zayv_status.split(', ')


active_objects_cosm = {
    'token': token_cosm_bi, 
    'report_id': 2,
    'fields': ['city_realtor', 'realtor_fio'],
    'filter_query': [
        {
            'field': 'city_realtor',
            'values': city_filter
        },
        {
            'field': 'status_ticket',
            'values': zayv_status
        },
        {
            'field': 'realtor_status',
            'values': rieltor_status
        },
        {
            'field': 'date_start_realtor',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
office_adress = 'Да(на основе задатка), Да(на основе показа), Да(на основе сделки), Да(настоящая)'
office_adress = office_adress.split(', ')
uchet_filter = 'Риэлтор'
uchet_filter = uchet_filter.split(', ')


podborki_cosm = {
    'token': token_cosm_bi, 
    'report_id': 9,
    'fields': ['city_realtor', 'rieltor_fio', 'first_date_selection'],
    'filter_query': [
        {
            'field': 'rieltor_city',
            'values': city_filter
        },
        {
            'field': 'address_offices_ticket',
            'values': office_adress 
        },
        {
            'field': 'uchit_m_r',
            'values': uchet_filter
        },
        {
            'field': 'rieltor_started_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'first_date_selection',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
otmetka_filter = 'Завершён'
otmetka_filter = otmetka_filter.split(', ')


pokazi_cosm = {
    'token': token_cosm_bi, 
    'report_id': 1,
    'fields': ['buy_realtor_city_name', 'realtor_buy'],
    'filter_query': [
        {
            'field': 'buy_realtor_city_name',
            'values': city_filter
        },
        {
            'field': 'buyer_show_completed',
            'values': otmetka_filter 
        },
        {
            'field': 'started_date_realtor',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'show_date',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
etap_filter = 'Консультация: личная, Консультация: телефонная, Консультирование, Отложена: Не отвечает после консультации'
etap_filter = otmetka_filter.split(', ')
status_filter = 'Активен'
status_filter = otmetka_filter.split(', ')
test_filter = 'Нет'
test_filter = test_filter.split(', ')


ik_cosm = {
    'token': token_cosm_bi, 
    'report_id': 7,
    'fields': ['realtor_city_name', 'realtor_fio', 'realtor_started_date', 'date_started_stage'],
    'filter_query': [
        {
            'field': 'realtor_status_user',
            'values': status_filter
        },
        {
            'field': 'stage_name',
            'values': etap_filter 
        },
        {
            'field': 'test_client',
            'values': test_filter 
        },
        {
            'field': 'realtor_city_name',
            'values': city_filter 
        },
        {
            'field': 'realtor_started_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_started_stage',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
etap_filter = 'Отправлена в банк(и)'
etap_filter = otmetka_filter.split(', ')
status_filter = 'Активен'
status_filter = otmetka_filter.split(', ')
test_filter = 'Нет'
test_filter = test_filter.split(', ')


ipotech_zayv_cosm = {
    'token': token_cosm_bi, 
    'report_id': 7,
    'fields': ['realtor_city_name', 'realtor_fio', 'realtor_started_date', 'date_started_stage'],
    'filter_query': [
        {
            'field': 'realtor_status_user',
            'values': status_filter
        },
        {
            'field': 'stage_name',
            'values': etap_filter 
        },
        {
            'field': 'test_client',
            'values': test_filter 
        },
        {
            'field': 'realtor_city_name',
            'values': city_filter 
        },
        {
            'field': 'realtor_started_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_started_stage',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
zadatok_status = 'В работе, Сделка, Сорвался(ошибочно)'
zadatok_status = zadatok_status.split(', ')
date_filter = '2023-08, 2023-09, 2023-10, 2023-11'
date_filter = date_filter.split(', ')


zadatki_cosm = {
    'token': token_cosm_bi, 
    'report_id': 11,
    'fields': ['realtor_city_name', 'realtor_fio', 'started_date', 'date_deposit'],
    'filter_query': [
        {
            'field': 'status_deposit',
            'values': zadatok_status
        },
        {
            'field': 'realtor_city_name',
            'values': city_filter 
        },
        {
            'field': 'realtor_started_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_deposit',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
city_filter = 'Бахчисарай, Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск, Севастополь'
city_filter = city_filter.split(', ')
zadatok_status = 'В работе, Сделка, Сорвался(ошибочно)'
zadatok_status = zadatok_status.split(', ')


pokup_prodaz_cosm = {
    'token': token_cosm_bi, 
    'report_id': 11,
    'fields': ['realtor_city_name', 'realtor_fio', 'started_date', 'date_deposit'],
    'filter_query': [
        {
            'field': 'status_deposit',
            'values': zadatok_status
        },
        {
            'field': 'realtor_city_name',
            'values': city_filter 
        },
        {
            'field': 'realtor_started_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'date_deposit',
            'values': [f'{y_year}-{y_month}-{y_day} TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
otdel_filter = 'Департамент продаж, Отдел продаж'
otdel_filter = otdel_filter.split(', ')
status_filter = 'Активен'
status_filter = status_filter.split(', ')
type_dolznosti = 'Специалист отдела продаж'
type_dolznosti = type_dolznosti.split(', ')
city_filter = 'Гатчина, Губкинский, Домодедово, Ижевск, Ишим, Когалым, Курган, Лангепас, Лянтор, Москва, Муравленко, Набережные Челны, Нижний Новгород, Нижний Тагил, Новый Волгоград, Новый Уренгой, Ноябрьск, Омск, Оренбург, Ростов-на-Дону, Самара, Санкт-Петербург, Саранск, Севастополь, Сургут, Тарко-Сале, Тобольск, Челябинск, Ялуторовск'
city_filter = city_filter.split(', ')
priority_status = 'Да'
priority_status = priority_status.split(', ')
type_group = 'Нет'
type_group = type_group.split(', ')


uchet_f1 ={
    'token': token_new_bi, 
    'report_id': 374,
    'fields': ['fio', 'type', 'position', 'department', 'manager_fio', 'start_date', 'date_dismiss', 'leave_reason', 'comment_leave',
               'user_status', 'hr_manager_fio', 'form_attraction', 'type_position', 'city'],
    'filter_query': [
        {
            'field': 'department',
            'values': otdel_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'type_position',
            'values': type_dolznosti
        },
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'status_priority',
            'values': priority_status
        },
        {
            'field': 'group_o_dashboard',
            'values': type_group
        }
    ],
    "rows": 10000,
    "start": 0
}



uchet_f1 = ExcelReview(uchet_f1)
uchet_f1.df = uchet_f1.get_table()
uchet_f1.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'f1_sotrudniki', 'a1')

In [ ]:
otdel_filter = 'Департамент продаж, Отдел продаж'
otdel_filter = otdel_filter.split(', ')
status_filter = 'Активен'
status_filter = status_filter.split(', ')
type_dolznosti = 'Специалист отдела продаж'
type_dolznosti = type_dolznosti.split(', ')
city_filter = 'Alanya, Antalya, Абакан, Актау, Актобе, Алапаевск, Алексеевка, Алматы, Алушта, Альметьевск, Анапа, Ангарск, Анжеро-Судженск, Арсеньев, Архангельск, Астана, Астрахань, Атырау, Ачинск, Аша, Баку, Балашов, Барнаул, Белгород, Белореченск, Бердск, Березовский (Кемеровская область), Бийск, Бишкек, Благовещенск, Боровичи, Брянск, Великий Новгород, Верхняя Салда, Владивосток, Владимир, Волгоград, Волжск, Вологда, Волхов, Воронеж, Воскресенск, Геленджик, Горно-Алтайск, Горячий Ключ, Грозный, Гусь-Хрустальный, Дербент, Дзержинск, Дмитров, Евпатория, Ейск, Екатеринбург, Ереван, Зеленогорск, Зеленодольск, Златоуст, Иваново, Ирбит, Иркутск, Йошкар-Ола, Казань, Калининград, Калуга, Камышлов, Каспийск, Кемерово, Кириши, Клин, Ковров, Кокшетау, Коломна, Комсомольск-на-Амуре, Короча, Корсаков, Коряжма, Костанай, Кострома, Котлас, Краснодар, Красноярск, Кызыл, Кыштым, Ленинск-Кузнецкий, Липецк, Луховицы, Майкоп, Махачкала, Междуреченск, Мелеуз, Миасс, Минск, Мичуринск, Можга, Муром, Надым, Назрань, Нальчик, Невинномысск, Нерчинск, Нефтекамск, Нефтеюганск, Нижневартовск, Новодвинск, Новокузнецк, Новороссийск, Новосибирск, Новоуральск, Норильск, Нягань, Обнинск, Октябрьский (Башкортостан), Орехово-Зуево, Орёл, Остров, Павлодар, Пенза, Первоуральск, Пермь, Пермь 2, Петрозаводск, Петропавловск, Петропавловск-Камчатский, Печоры, Подольск, Пойковский пгт., Покров, Прокопьевск, Прохладный, Псков, Пыть-Ях, Пятигорск, Ракитное, Раменское, Московская обл., Рудный, Саки, Салават, Салехард, Самарканд, Сарапул, Саратов, Сатка, Северодвинск, Семей, Симферополь, Смоленск, Сосновоборск, Сочи, Ставрополь, Старый Оскол, Стерлитамак, Сыктывкар, Таганрог, Тамбов, Ташкент, Тверь, Темрюк, Тольятти, Томск, Троицк, Туапсе, Туймазы, Тула, Туринск, Улан-Удэ, Ульяновск, Уральск, Усинск, Усть-Каменогорск, Уфа, Ухта, Феодосия, Хабаровск, Ханты-Мансийск, Чебаркуль, Череповец, Черкесск, Черногорск, Чита, Шадринск, Шымкент, Южно-Сахалинск, Южноуральск, Юрга, Якутск, Ялта, Ярославль, Ярцево'
city_filter = city_filter.split(', ')
priority_status = 'Да'
priority_status = priority_status.split(', ')
type_group = 'Нет'
type_group = type_group.split(', ')


uchet_f2 ={
    'token': token_new_bi, 
    'report_id': 374,
    'fields': ['fio', 'type', 'position', 'department', 'manager_fio', 'start_date', 'date_dismiss', 'leave_reason', 'comment_leave',
               'user_status', 'hr_manager_fio', 'form_attraction', 'type_position', 'city', 'add_work_address'],
    'filter_query': [
        {
            'field': 'department',
            'values': otdel_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'type_position',
            'values': type_dolznosti
        },
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'status_priority',
            'values': priority_status
        },
        {
            'field': 'group_o_dashboard',
            'values': type_group
        }
    ],
    "rows": 10000,
    "start": 0
}



uchet_f2 = ExcelReview(uchet_f2)
uchet_f2.df = uchet_f2.get_table()
#uchet_f2.load_full_table('1plph8ILPblP1G_Mnk58VP02pf2jZ7eiTtgb2IVi5AFo', 'f2_sotrudniki', 'a1')

In [ ]:
otdel_filter = 'Департамент продаж, Отдел продаж'
otdel_filter = otdel_filter.split(', ')
status_filter = 'Активен'
status_filter = status_filter.split(', ')
type_dolznosti = 'Менеджер отдела продаж, Специалист отдела продаж'
type_dolznosti = type_dolznosti.split(', ')
sotr_filer = 'agent'
sotr_filer = sotr_filer.split(', ')
city_filter = 'Тюмень'
city_filter = city_filter.split(', ')
priority_status = 'Да'
priority_status = priority_status.split(', ')
type_group = 'Нет'
type_group = type_group.split(', ')


uchet_tymen ={
    'token': token_new_bi, 
    'report_id': 374,
    'fields': ['fio', 'position', 'department', 'manager_fio', 'start_date',
               'user_status', 'hr_manager_fio', 'city', 'add_work_address', 'rop'],
    'filter_query': [
        {
            'field': 'department',
            'values': otdel_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'type_position',
            'values': type_dolznosti
        },
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'status_priority',
            'values': priority_status
        },
        {
            'field': 'group_o_dashboard',
            'values': type_group
        },
        {
            'field': 'type',
            'values': sotr_filer
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
otdel_filter = 'Департамент продаж, Отдел продаж'
otdel_filter = otdel_filter.split(', ')
status_filter = 'Активен'
status_filter = status_filter.split(', ')
type_dolznosti = 'Специалист отдела продаж'
type_dolznosti = type_dolznosti.split(', ')
city_filter = 'Бахчисарай, Севастополь'
city_filter = city_filter.split(', ')
priority_status = 'Да'
priority_status = priority_status.split(', ')
type_group = 'Нет'
type_group = type_group.split(', ')


uchet_cosm_f1 ={
    'token': token_new_bi, 
    'report_id': 3,
    'fields': ['fio', 'type', 'position', 'department', 'manager_fio', 'start_date', 'date_dismiss', 'leave_reason', 'comment_leave',
               'user_status', 'hr_manager_fio', 'form_attraction', 'type_position', 'city'],
    'filter_query': [
        {
            'field': 'department',
            'values': otdel_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'type_position',
            'values': type_dolznosti
        },
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'status_priority',
            'values': priority_status
        },
        {
            'field': 'group_o_dashboard',
            'values': type_group
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
otdel_filter = 'Департамент продаж, Отдел продаж'
otdel_filter = otdel_filter.split(', ')
status_filter = 'Активен'
status_filter = status_filter.split(', ')
type_dolznosti = 'Специалист отдела продаж'
type_dolznosti = type_dolznosti.split(', ')
city_filter = 'Владимир, Вологда, Геленджик, Гусь-Хрустальный, Зеленогорск'
city_filter = city_filter.split(', ')
priority_status = 'Да'
priority_status = priority_status.split(', ')
type_group = 'Нет'
type_group = type_group.split(', ')


uchet_cosm_f2 ={
    'token': token_new_bi, 
    'report_id': 3,
    'fields': ['fio', 'type', 'position', 'department', 'manager_fio', 'start_date', 'date_dismiss', 'leave_reason', 'comment_leave',
               'user_status', 'hr_manager_fio', 'form_attraction', 'type_position', 'city'],
    'filter_query': [
        {
            'field': 'department',
            'values': otdel_filter
        },
        {
            'field': 'start_date',
            'values': [f'{s_year}-{s_month}-01 TO {c_year}-{c_month}-{c_day}'],
            'type': 'date'
        },
        {
            'field': 'user_status',
            'values': status_filter
        },
        {
            'field': 'type_position',
            'values': type_dolznosti
        },
        {
            'field': 'city',
            'values': city_filter
        },
        {
            'field': 'status_priority',
            'values': priority_status
        },
        {
            'field': 'group_o_dashboard',
            'values': type_group
        }
    ],
    "rows": 10000,
    "start": 0
}

In [ ]:
#проверить работу космических таблиц - не работат бд космоса

In [ ]:
#load_by_day - обновляет таблицу на ежедневной основе
#load_full_table - загружает первый день в начале месяца
#для make_sort_table нужны поля из таблицы, которые влияют на merge

In [ ]:
#выкачать всю таблицу, сделать merge со всей таблицей
#т.к. в таблице сгрупированные данные выкачиваться и закачиваться будет не долго